In [3]:
import datetime


from cogment_lab.actors import RandomActor, ConstantActor
from cogment_lab.envs.pettingzoo import ParallelEnvironment
from cogment_lab.process_manager import Cogment
from cogment_lab.utils.runners import process_cleanup


In [4]:
# Cleans up potentially hanging background processes from previous runs
process_cleanup()

Processes terminated successfully.


In [5]:
logpath = f"logs/logs-{datetime.datetime.now().isoformat()}"

cog = Cogment(log_dir=logpath)

print(logpath)

logs/logs-2024-01-15T21:53:56.839732


In [6]:
# Launch an environment in a subprocess

cenv = ParallelEnvironment(env_path="pettingzoo.butterfly.cooperative_pong_v5.parallel_env",
                      render=True)

await cog.run_env(env=cenv, 
                  env_name="pong",
                  port=9011, 
                  log_file="env.log")


True

In [7]:
# Launch two dummy actors in subprocesses

random_actor = RandomActor(cenv.env.action_space("paddle_0"))
constant_actor = ConstantActor(1)

await cog.run_actor(actor=random_actor, 
                    actor_name="random", 
                    port=9021, 
                    log_file="actor-random.log")

await cog.run_actor(actor=constant_actor,
                    actor_name="constant",
                    port=9022,
                    log_file="actor-constant.log")


True

In [8]:
# Check what's running

cog.processes

{'pong': <SpawnProcess name='SpawnProcess-1' pid=54386 parent=54362 started>,
 'random': <SpawnProcess name='SpawnProcess-2' pid=54391 parent=54362 started>,
 'constant': <SpawnProcess name='SpawnProcess-3' pid=54394 parent=54362 started>}

In [11]:
PONG_ACTIONS = ["no-op", "ArrowUp", "ArrowDown"]

actions = PONG_ACTIONS
await cog.run_web_ui(actions=actions, log_file="human.log", fps=60)

True

In [9]:
# Get data from a random + random trial
# You can change the values in `actor_impls` between `web_ui`, `random`, and `constant` to see the different behaviors

trial_id = await cog.start_trial(
    env_name="pong",
    session_config={"render": True},
    actor_impls={
        "paddle_0": "random",
        "paddle_1": "constant",
    },
)

data = await cog.get_trial_data(trial_id)


In [10]:
data["paddle_0"].actions

array([2, 2, 2, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 0,
       1, 2])